In [2]:
import csv
from rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS, OWL, XSD
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore

def create_perspectivisation_graph(csv_file, endpoint_url):
    """
    Reads the CSV of LGBTQ+ teen TV characters and maps each row to your
    Perspectivisation ontology. Then uploads to a SPARQL endpoint (e.g. Blazegraph).
    """

    # 1) SPARQL endpoint
    #    Example: http://localhost:9999/blazegraph/sparql
    endpoint = "http://localhost:9999/blazegraph/sparql"

    # 2) Ontology namespace (perspectivisation.owl) plus an example namespace for new individuals
    PERSPECT = Namespace("http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#")
    EX       = Namespace("http://example.org/resource/")

    # 3) Create a new RDF graph
    g = Graph()

    # 4) Bind prefixes (for nicer serialization)
    g.bind("persp", PERSPECT)
    g.bind("ex",    EX)

    # --------------------------------------------------------------------------
    # (Optional) Re-declare some classes and properties if your store is NOT
    # already loaded with your ontology. If it is, you can remove these lines.
    # --------------------------------------------------------------------------
    g.add((PERSPECT.FairRepresentation,    RDF.type, OWL.Class))
    g.add((PERSPECT.UnfairRepresentation,  RDF.type, OWL.Class))
    g.add((PERSPECT.DetailedPortrayal,     RDF.type, OWL.Class))
    g.add((PERSPECT.SuperficialPortrayal,  RDF.type, OWL.Class))
    g.add((PERSPECT.AuthenticCharacterization, RDF.type, OWL.Class))
    g.add((PERSPECT.CharacterTransformation, RDF.type, OWL.Class))
    g.add((PERSPECT.DynamicRole, RDF.type, OWL.Class))
    g.add((PERSPECT.StaticRole,  RDF.type, OWL.Class))
    g.add((PERSPECT.StereotypicalCharacterization, RDF.type, OWL.Class))
    g.add((PERSPECT.PlotResolution, RDF.type, OWL.Class))
    g.add((PERSPECT.IdeologicalChallenge, RDF.type, OWL.Class))
    g.add((PERSPECT.ProfitAndMassAppeal,  RDF.type, OWL.Class))

    # Classes from the ontology's "roles"
    g.add((PERSPECT.Attitude,    RDF.type, OWL.Class))
    g.add((PERSPECT.Lens,        RDF.type, OWL.Class))
    g.add((PERSPECT.Cut,         RDF.type, OWL.Class))
    g.add((PERSPECT.Eventuality, RDF.type, OWL.Class))
    g.add((PERSPECT.Conceptualizer, RDF.type, OWL.Class))
    g.add((PERSPECT.Perspectivisation, RDF.type, OWL.Class))

    # Punned properties
    g.add((PERSPECT.Attitude,    RDF.type, OWL.ObjectProperty))
    g.add((PERSPECT.Lens,        RDF.type, OWL.ObjectProperty))
    g.add((PERSPECT.Cut,         RDF.type, OWL.ObjectProperty))
    g.add((PERSPECT.Eventuality, RDF.type, OWL.ObjectProperty))
    g.add((PERSPECT.Conceptualiser, RDF.type, OWL.ObjectProperty))
    # etc. if desired

    # For storing textual columns, define some data properties in our EX namespace
    # (You can also use rdfs:comment or define them in your ontology if you prefer.)
    hasShowTitle    = EX.hasShowTitle
    hasJustification= EX.hasJustification
    hasImpact       = EX.hasImpact
    hasAffirmation  = EX.hasAffirmation
    hasNormalization= EX.hasNormalization
    hasGBF          = EX.hasGayBestFriend
    hasTragicTrope  = EX.hasTragicTrope
    hasTokenism     = EX.hasTokenism
    hasPlotDesc     = EX.hasPlotResolutionDesc

    for dp in [hasShowTitle, hasJustification, hasImpact, hasAffirmation,
               hasNormalization, hasGBF, hasTragicTrope, hasTokenism, hasPlotDesc]:
        g.add((dp, RDF.type, OWL.DatatypeProperty))
        g.add((dp, RDFS.range, URIRef(XSD.string)))  # store them as xsd:string

    # 5) Open the CSV and process each row
    with open("new dataset(Foglio1).csv", mode='r', encoding='utf-8', errors="replace") as f:
        reader = csv.DictReader(f)
        
        row_num = 0
        for row in reader:
            row_num += 1

            # ---------------------------
            # Extract columns from CSV
            # ---------------------------
            char_name     = row["Character Name"].strip()
            show_title    = row["Show Title"].strip()
            rep_type      = row["Representation Type"].strip()  # "Fair Representation" / "Unfair Representation"
            justification = row["Justification for Classification"].strip()
            impact        = row["LGBTQ Community Impact"].strip()
            portrayal     = row["Portrayal"].strip()            # "Detailed" / "Superficial"
            auth_char     = row["Authentic Characterization"].strip()  # "Yes" / "No"
            affirm_id     = row["Affirmation of Identity"].strip()      # "Yes"/"No"
            normalize_rel = row["Normalization of Relationship"].strip()# "Yes"/"No"
            dynamic_role  = row["Dynamic Role"].strip()                 # "Yes"/"No"
            static_role   = row["Static Role"].strip()                  # "Yes"/"No"
            stereo_char   = row["Stereotypical Characterization"].strip() # "Yes"/"No"
            gay_bf        = row["Gay Best Friend"].strip()              # "Yes"/"No"
            tragic_trope  = row["Tragic Trope"].strip()                 # "Yes"/"No"
            tokenism      = row["Tokenism"].strip()                     # "Yes"/"No"
            char_trans    = row["Character Transformation"].strip()     # possibly text
            plot_res      = row["Plot Resolution"].strip()              # possibly text
            lens_orient   = row["Lens Orientation"].strip()             # "Ideological Challenge" / "Profit and Mass Appeal"

            # ------------------------------------------
            # Create a main Perspectivisation for this row
            # plus role individuals (Cut, Lens, Attitude, etc.)
            # ------------------------------------------
            # e.g., ex:persp_1, ex:cut_1, ex:lens_1, ex:att_1, ...
            persp_uri = EX[f"Perspectivisation_{row_num}"]
            cut_uri   = EX[f"Cut_{row_num}"]
            lens_uri  = EX[f"Lens_{row_num}"]
            att_uri   = EX[f"Attitude_{row_num}"]
            ev_uri    = EX[f"Eventuality_{row_num}"]
            con_uri   = EX[f"Conceptualizer_{row_num}"]
            # (We won’t create a "Background" because CSV has no background column, 
            #  but you could if needed.)

            # Type them according to the ontology roles
            g.add((persp_uri, RDF.type, PERSPECT.Perspectivisation))
            g.add((cut_uri,   RDF.type, PERSPECT.Cut))
            g.add((lens_uri,  RDF.type, PERSPECT.Lens))
            g.add((att_uri,   RDF.type, PERSPECT.Attitude))
            g.add((ev_uri,    RDF.type, PERSPECT.Eventuality))
            g.add((con_uri,   RDF.type, PERSPECT.Conceptualizer))

            # Link them: Perspectivisation -> {Cut, Lens, Attitude, Eventuality, Conceptualiser}
            g.add((persp_uri, PERSPECT.Cut,         cut_uri))
            g.add((persp_uri, PERSPECT.Lens,        lens_uri))
            g.add((persp_uri, PERSPECT.Attitude,    att_uri))
            g.add((persp_uri, PERSPECT.Eventuality, ev_uri))
            g.add((persp_uri, PERSPECT.Conceptualiser, con_uri))

            # We also say that the Attitude is "toward" the Cut:
            g.add((att_uri, PERSPECT.toward, cut_uri))

            # ------------------------------------------
            # Multi-type the CUT individual based on CSV
            # e.g. if rep_type is "Fair Representation" => type :FairRepresentation
            # ------------------------------------------
            if rep_type.lower().startswith("fair"):
                g.add((cut_uri, RDF.type, PERSPECT.FairRepresentation))
                # Optionally set "accuracyOfRepresentation" to "High"
                # only if you want that property automatically:
                g.add((cut_uri, PERSPECT.accuracyOfRepresentation, Literal("High")))
            else:
                g.add((cut_uri, RDF.type, PERSPECT.UnfairRepresentation))
                g.add((cut_uri, PERSPECT.accuracyOfRepresentation, Literal("Low")))

            # Portrayal => DetailedPortrayal or SuperficialPortrayal
            if portrayal.lower() == "detailed":
                g.add((cut_uri, RDF.type, PERSPECT.DetailedPortrayal))
            else:
                g.add((cut_uri, RDF.type, PERSPECT.SuperficialPortrayal))

            # Authentic Characterization => if yes => add type
            if auth_char.lower() == "yes":
                g.add((cut_uri, RDF.type, PERSPECT.AuthenticCharacterization))

            # Dynamic Role => if yes => add type
            if dynamic_role.lower() == "yes":
                g.add((cut_uri, RDF.type, PERSPECT.DynamicRole))

            # Static Role => if yes => add type
            if static_role.lower() == "yes":
                g.add((cut_uri, RDF.type, PERSPECT.StaticRole))

            # Stereotypical Characterization => if yes => add type
            if stereo_char.lower() == "yes":
                g.add((cut_uri, RDF.type, PERSPECT.StereotypicalCharacterization))

            # Character Transformation => if not empty => type CharacterTransformation
            if char_trans.strip():
                g.add((cut_uri, RDF.type, PERSPECT.CharacterTransformation))

            # Plot Resolution => if not empty => type PlotResolution
            if plot_res.strip():
                g.add((cut_uri, RDF.type, PERSPECT.PlotResolution))

            # -----------------------------------------------------
            # Lens orientation => type the LENS as IdeologicalChallenge or ProfitAndMassAppeal
            # -----------------------------------------------------
            if lens_orient.lower().startswith("ideological"):
                g.add((lens_uri, RDF.type, PERSPECT.IdeologicalChallenge))
            else:
                g.add((lens_uri, RDF.type, PERSPECT.ProfitAndMassAppeal))

            # ------------------------------------------
            # Store textual data (Character Name, Show, Justifications, etc.)
            # on the "Cut" or a relevant node
            # ------------------------------------------
            # For clarity, let’s store the character name and show title on the Cut
            # as label + a custom property:
            g.add((cut_uri, RDFS.label, Literal(char_name, datatype=XSD.string)))
            g.add((cut_uri, hasShowTitle, Literal(show_title, datatype=XSD.string)))

            # Justification for Classification => store as property
            if justification:
                g.add((cut_uri, hasJustification, Literal(justification, datatype=XSD.string)))

            # LGBTQ Community Impact => store as property
            if impact:
                g.add((cut_uri, hasImpact, Literal(impact, datatype=XSD.string)))

            # Affirmation of Identity => store as property
            if affirm_id:
                g.add((cut_uri, hasAffirmation, Literal(affirm_id, datatype=XSD.string)))

            # Normalization of Relationship => store as property
            if normalize_rel:
                g.add((cut_uri, hasNormalization, Literal(normalize_rel, datatype=XSD.string)))

            # Gay Best Friend => store as property
            if gay_bf:
                g.add((cut_uri, hasGBF, Literal(gay_bf, datatype=XSD.string)))

            # Tragic Trope => store as property
            if tragic_trope:
                g.add((cut_uri, hasTragicTrope, Literal(tragic_trope, datatype=XSD.string)))

            # Tokenism => store as property
            if tokenism:
                g.add((cut_uri, hasTokenism, Literal(tokenism, datatype=XSD.string)))

            # Plot Resolution => store the text description
            if plot_res:
                g.add((cut_uri, hasPlotDesc, Literal(plot_res, datatype=XSD.string)))

    # --------------------------------------------------------------------------
    # 6) Upload everything to the SPARQL endpoint
    # --------------------------------------------------------------------------
    store = SPARQLUpdateStore()
    store.open((endpoint, endpoint))  # pass as (query, update)

    for triple in g.triples((None, None, None)):
        store.add(triple)

    store.close()
    print(f"Uploaded {len(g)} triples to {endpoint}.")

# ------------------------------------------------------------------------------
# Example usage
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # Replace these with your actual CSV path and your SPARQL endpoint
    csv_file_path = "new dataset(Foglio1).csv"  # CSV with the columns you provided
    blazegraph_sparql_endpoint = "http://localhost:9999/blazegraph/sparql"

    create_perspectivisation_graph(csv_file_path, blazegraph_sparql_endpoint)


Uploaded 937 triples to http://localhost:9999/blazegraph/sparql.
